In [2]:
import folium
import webbrowser
from customtkinter import CTkImage 
import requests
from geopy.distance import geodesic
import customtkinter as ctk
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image as PILImage, ImageTk
import numpy as np
import tensorflow as tf
import mysql.connector
import time
import webbrowser

In [ ]:
model = tf.keras.models.load_model('/Users/owaiskhan/Downloads/Lung Disease Dataset/best_1_lung_disease_model.keras')
CLASS_LABELS = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']

# OpenCage API Key 
API_KEY = "5fa7428842fd4e2c95d7615297268a95"

def open_chatbot():
    result_text = result_label.cget("text")
    if any(pred_class in result_text for pred_class in CLASS_LABELS):
        predicted_disease = result_text.split(", ")[1]  # Extract disease name
        return chatbot_window(predicted_disease)
    else:
        messagebox.showwarning("No Prediction", "Run a prediction first.")
        return

# def chatbot_response(user_query, predicted_disease):
#     query = user_query.lower()
#     disease_data = fetch_disease_info_from_db(predicted_disease)

#     if not disease_data:
#         return "Sorry, I don't have information about that disease."

#     if "symptom" in query:
#         return "Common symptoms include: " + disease_data["symptoms"] if disease_data["symptoms"] else "No symptoms available."
#     elif "cure" in query or "treatment" in query:
#         return "Cure/Treatment: " + disease_data["cure"]
#     elif "prescription" in query or "medicine" in query or "medication" in query:
#         return "Prescription advice: " + disease_data["prescription"]
#     elif "info" in query or "what is" in query or "about" in query:
#         return "Here's some information: " + disease_data["info"]
#     else:
#         return "I'm here to help! You can ask about the symptoms, cure, prescription, or general info about the disease."


def fetch_disease_info_from_db(predicted_disease):
    try:
        connection = mysql.connector.connect(
            host="localhost",  # or your DB host
            user="root",
            password="Owais$123",
            database="lung_disease_detection"
        )
        cursor = connection.cursor(dictionary=True)
        cursor.execute("SELECT * FROM diseases WHERE name = %s", (predicted_disease,))
        row = cursor.fetchone()
        return row
    except Exception as e:
        print("Error fetching disease info:", e)
        return None
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()


def chatbot_window(predicted_disease):
    
    disease_data = fetch_disease_info_from_db(predicted_disease) or {}

    chatbot_window = ctk.CTkToplevel()
    chatbot_window.title("HealthBot - Lung Disease Assistant")
    chatbot_window.geometry("500x600")

    chat_history = ctk.CTkTextbox(chatbot_window, width=480, height=400, wrap="word")
    chat_history.pack(pady=20)
    chat_history.insert(tk.END, f"🤖 Hello! I can help you with information about {predicted_disease}.\n")
    chat_history.insert(tk.END, "Ask me about symptoms, cure, prescriptions, or general info.\n\n")
    chat_history.configure(state="disabled")

    # --- Quick-access buttons frame ---
    btn_frame = ctk.CTkFrame(chatbot_window)
    btn_frame.pack(pady=(0,10))

    def bot_reply(text):
        chat_history.configure(state="normal")
        chat_history.insert(tk.END, f"🤖 {text}\n\n")
        chat_history.configure(state="disabled")
        chat_history.see(tk.END)

    def show_symptoms():
        sym = disease_data.get("symptoms", "")
        bot_reply("Common symptoms include: " + (sym or "No data available."))

    def show_cure():
        bot_reply("Treatment/Cure: " + disease_data.get("cure", "No data available."))

    def show_info():
        bot_reply("About the disease: " + disease_data.get("info", "No data available."))

    def show_prescription():
        bot_reply("Prescription advice: " + disease_data.get("prescription", "No data available."))

        # Create buttons
    for (text, cmd) in [
        ("Symptoms", show_symptoms),
        ("Cure",     show_cure),
        ("Info",     show_info),
        ("Prescription", show_prescription),
    ]:
        ctk.CTkButton(btn_frame, text=text, command=cmd, width=110).pack(side="left", padx=5)

    
    user_input = ctk.CTkEntry(chatbot_window, width=400, placeholder_text="Type your question here...")
    user_input.pack(pady=10)

    def send_message():
        query = user_input.get()
        if query.strip() == "":
            return
        user_input.delete(0, tk.END)

        chat_history.configure(state="normal")
        chat_history.insert(tk.END, f"🧑 You: {query}\n")

        url = f"https://www.google.com/search?q={query.replace(' ', '+')}"
        webbrowser.open_new_tab(url)
        bot_reply(f"I've opened a Google search for “{query}” in your browser.")

        # response = chatbot_response(query, predicted_disease)
        # chat_history.insert(tk.END, f"🤖 Bot: {response}\n\n")
        # chat_history.configure(state="disabled")
        # chat_history.see(tk.END)

    send_btn = ctk.CTkButton(chatbot_window, text="Send", command=send_message)
    send_btn.pack()


def show_map():
    user_name = name_entry.get().strip()
    if not user_name:
        messagebox.showwarning("Name Required", "Please enter your name first.")
        return
    user_data = get_user_location(user_name)
    if not user_data:
        messagebox.showwarning("User Not Found", "No profile found. Please create a profile first.")
        return

    user_lat, user_lon = user_data
    
    result_text = result_label.cget("text")
    if any(pred_class in result_text for pred_class in CLASS_LABELS):
        predicted_disease = result_text.split(", ")[1]  # Extract disease name
    else:
        messagebox.showwarning("No Prediction", "Run a prediction first.")
        return
    
    map_path = generate_hospital_map(user_lat, user_lon, predicted_disease)
    if map_path:
        webbrowser.open(map_path)  # Open in a browser

    
def generate_hospital_map(user_lat, user_lon, predicted_disease):

    user_name = name_entry.get().strip()
    if not user_name:
        messagebox.showwarning("Invalid Input", "Please enter your name.")
        return
        
    connection = get_db_connection()
    cursor = connection.cursor()
    
    cursor.execute("SELECT location FROM users WHERE user_name = %s", (user_name,))
    location = cursor.fetchone()
    
    cursor.close()
    connection.close()
    
    # connection = get_db_connection()
    # cursor = connection.cursor()
    # cursor.execute("SELECT name, latitude, longitude FROM hospitals WHERE specialties  = %s",(disease_name,))
    # hospitals = cursor.fetchall()
    # cursor.close()
    # connection.close()

    hospitals = get_hospitals_for_disease(predicted_disease)
    
    if not hospitals:
        return "No hospitals found for this disease."
    
    # Compute distances
    hospital_distances = []
    for hospital in hospitals:
        name, address, lat, lon, doc_name, contact = hospital
        distance = calculate_distance((user_lat, user_lon), (lat, lon))
        hospital_distances.append((name, address, lat, lon, distance))
    
    # Sort hospitals by distance and get the top 4-5
    hospital_distances.sort(key=lambda x: x[4])
    top_hospitals = hospital_distances[:5]
    
    
    if not top_hospitals:
        messagebox.showinfo("No Hospitals Found", "No hospitals available for this disease.")
        return None
    m = folium.Map(location=[user_lat, user_lon], zoom_start=12)
    folium.Marker(
        location=[user_lat, user_lon],
        popup=f"Your Location - {location}                 ",
        icon=folium.Icon(color="red", icon="home")
    ).add_to(m)

    for name, address, lat, lon, distance in top_hospitals:
        folium.Marker(
            location=[lat, lon],
            popup=f"{name},{address}", 
            icon=folium.Icon(color="blue", icon="plus")
        ).add_to(m)
    
    map_path = "hospital_map.html"
    m.save(map_path)
    return map_path




def toggle_theme():
    if theme_switch.get() == 0:  # If switch is ON
        ctk.set_appearance_mode("dark")
        theme_switch.configure(text="Dark Mode")  # Update switch text
    else:  # If switch is OFF
        ctk.set_appearance_mode("light")
        theme_switch.configure(text="Light Mode")  # Update switch text

def show_loading():
    result_label.configure(text="Processing... Please wait.")
    win.update_idletasks()

    
def show_recommendations():
    user_name = name_entry.get().strip()
    if not user_name:
        messagebox.showwarning("Invalid Input", "Please enter your name.")
        return
    
    result_text = result_label.cget("text")
    if any(pred_class in result_text for pred_class in CLASS_LABELS):
        predicted_disease = result_text.split(", ")[1]  # Extract disease name
    else:
        messagebox.showwarning("No Prediction", "Run a prediction first.")
        return

    recommendations = recommend_hospitals(user_name, predicted_disease)
    
    if isinstance(recommendations, str):
        hospital_label.configure(text=recommendations)
    else:
        hospital_info = "Recommended Hospitals:\n\n"
        for i, (name, address, doc_name, contact, distance) in enumerate(recommendations, start=1):
            hospital_info += f"{i}. {name},\nDoctor Name: {doc_name},\n Contact: {contact}, {address} ({distance:.2f} km)\n\n"
        hospital_label.configure(text=hospital_info)
        
def recommend_hospitals(user_name, predicted_disease):
    user_lat, user_lon = get_user_location(user_name)
    
    if user_lat is None or user_lon is None:
        return "User location not found."

    hospitals = get_hospitals_for_disease(predicted_disease)
    
    if not hospitals:
        return "No hospitals found for this disease."
    
    # Compute distances
    hospital_distances = []
    for hospital in hospitals:
        name, address, lat, lon, doc_name, contact = hospital
        distance = calculate_distance((user_lat, user_lon), (lat, lon))
        hospital_distances.append((name, address, doc_name, contact, distance))
    
    # Sort hospitals by distance and get the top 4-5
    hospital_distances.sort(key=lambda x: x[4])
    top_hospitals = hospital_distances[:5]
    
    return top_hospitals

def calculate_distance(user_coords, hospital_coords):
    return geodesic(user_coords, hospital_coords).kilometers

def get_hospitals_for_disease(predicted_disease):
    connection = get_db_connection()
    cursor = connection.cursor()
    
    cursor.execute("SELECT name, address, latitude, longitude, doctor_name, contact_info FROM hospitals WHERE specialties = %s", (predicted_disease,))
    hospitals = cursor.fetchall()
    
    cursor.close()
    connection.close()
    
    return hospitals

def get_user_location(user_name):
    connection = get_db_connection()
    cursor = connection.cursor()
    
    cursor.execute("SELECT latitude, longitude FROM users WHERE user_name = %s", (user_name,))
    result = cursor.fetchone()
    
    cursor.close()
    connection.close()
    
    if result:
        return result  # (latitude, longitude)
    else:
        return None, None

# Function to get latitude & longitude from an address
def geocode_address(address):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    
    if data['results']:
        latitude = data['results'][0]['geometry']['lat']
        longitude = data['results'][0]['geometry']['lng']
        return latitude, longitude
    else:
        return None, None


def resize_image(image_path, max_width, max_height):
    img = PILImage.open(image_path)
    
    # Get original dimensions
    width, height = img.size

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Determine the new size maintaining the aspect ratio
    if width > height:
        new_width = min(max_width, width)
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = min(max_height, height)
        new_width = int(new_height * aspect_ratio)

    size = (new_width, new_height)
    return size

def get_db_connection():
    return mysql.connector.connect(
        host="localhost", 
        user="root",  
        password="Owais$123",  
        database="lung_disease_detection"
    )

# Preprocess image function
def preprocess_image(image_path):
    try:
        image = PILImage.open(image_path)
    except Exception as e:
        print(f"Error opening image: {e}")
        return None

    image = image.resize((299, 299))
    image = np.array(image) / 255.0
    if len(image.shape) == 2:  
        image = np.stack((image,) * 3, axis=-1)
    image = np.expand_dims(image, axis=0)
    return image

# Function to create a new user profile
def create_profile():
    user_name = name_entry.get().strip()
    user_location = location_entry.get().strip()
    
    if not user_name:
        messagebox.showwarning("Invalid Name", "Please enter a valid username.")
        return
    
    if not user_location:
        messagebox.showwarning("Invalid Location", "Please enter your location.")
        return
    
    latitude, longitude = geocode_address(user_location)
    
    if latitude is None or longitude is None:
        messagebox.showerror("Location Error", "Could not fetch location data. Please enter a valid address.")
        return
    
    connection = get_db_connection()
    cursor = connection.cursor()
    
    # Check if user already exists
    cursor.execute("SELECT * FROM users WHERE user_name = %s", (user_name,))
    result = cursor.fetchone()
    
    if result:
        messagebox.showwarning("Profile Exists", "This username already exists.")
    else:
        # Insert user data with location
        cursor.execute(
            "INSERT INTO users (user_name, location, latitude, longitude) VALUES (%s, %s, %s, %s)",
            (user_name, user_location, latitude, longitude)
        )
        connection.commit()
        messagebox.showinfo("Profile Created", f"Profile for {user_name} created successfully!")
    
    cursor.close()
    connection.close()


# Function to get user ID from name
def get_user_id(user_name):
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT user_id FROM users WHERE user_name = %s", (user_name,))
    result = cursor.fetchone()
    cursor.close()
    connection.close()
    return result[0] if result else None

# Function to save image history to database
def save_history(user_name, image_path, predicted_class):
    user_id = get_user_id(user_name)
    check_query = "SELECT * FROM history WHERE user_id = %s AND image_path = %s AND predicted_class = %s"
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute(check_query, (user_id, image_path, predicted_class))
    existing_entry = cursor.fetchone()
    if not existing_entry:
        if user_id:
            connection = get_db_connection()
            cursor = connection.cursor()
            cursor.execute(
                "INSERT INTO history (user_id, image_path, predicted_class) VALUES (%s, %s, %s)",
                (user_id, image_path, predicted_class)
            )
            connection.commit()
            cursor.close()
            connection.close()
    else:
        messagebox.showinfo("Redundancy", "Duplicate entry detected. Not inserting")
        # messagebox.showinfo("Duplicate entry detected. Not inserting.")
        cursor.close()
        connection.close()
        
# Function to view user history
def view_history():
    user_name = name_entry.get().strip()
    user_id = get_user_id(user_name)
    
    if user_id:
        connection = get_db_connection()
        cursor = connection.cursor()
        cursor.execute("SELECT image_path, predicted_class FROM history WHERE user_id = %s", (user_id,))
        history = cursor.fetchall()
        
        if history:
            history_text = f"History for {user_name}:\n\n"
            for i, (file_path, predicted_class) in enumerate(history):
                history_text += f"{i+1}. Image: {file_path} -> Predicted: {predicted_class}\n"
            history_label.configure(text=history_text)
        else:
            messagebox.showinfo("No History", "No history found for this user.")
        
        cursor.close()
        connection.close()
    else:
        messagebox.showinfo("No Profile", "User profile not found. Please create a profile first.")


def predict_class():
    show_loading()
    user_name = name_entry.get().strip()
    
    # Check if user has entered a name
    if not user_name:
        messagebox.showwarning("Name Required", "Please enter your name first.")
        return
    
    # Check if the user exists in the database
    user_id = get_user_id(user_name)
    if not user_id:
        messagebox.showwarning("Profile Not Found", "No profile found. Please create a profile first.")
        return

    file_path = filedialog.askopenfilename(initialdir="C:/Users/Administrator/Downloads", title="Select X-Ray Image")
    
    if file_path:
        image = preprocess_image(file_path)
        prediction = model.predict(image)
        class_label = np.argmax(prediction, axis=1)[0]

        # Show the image on the GUI
        img = PILImage.open(file_path)
        img = CTkImage(img, size = resize_image(file_path,200,200))
        image_label.configure(image=img)
        image_label.image = img
        result_label.configure(text=f"Predicted Class: {class_label} , {CLASS_LABELS[class_label]}")

        # Save history if the user has a profile
        save_history(user_name, file_path, CLASS_LABELS[class_label])


# Function for guest mode (no profile)
def guest_mode():
    show_loading()
    file_path = filedialog.askopenfilename(initialdir="C:/Users/Administrator/Downloads", title="Select X-Ray Image")

    if file_path:
        image = preprocess_image(file_path)
        prediction = model.predict(image)
        class_label = np.argmax(prediction, axis=1)[0]

        # Show the image on the GUI
        img = PILImage.open(file_path)
        img = CTkImage(img, size = resize_image(file_path,200,200))
        image_label.configure(image=img)
        image_label.image = img
        
        result_label.configure(text=f"Predicted Class: {class_label} , {CLASS_LABELS[class_label]}")


win = ctk.CTk()
win.title("Lung Disease Detection")
win.geometry("900x650")
# win.attributes('-fullscreen', True)
# win.overrideredirect(False)
# win.resizable(False, False)

# screen_width = win.winfo_screenwidth()
# screen_height = win.winfo_screenheight()

# # Set the window size to full screen by using screen dimensions
# win.geometry(f"{screen_width}x{screen_height}+0+0")


ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("dark-blue")

# Main frame with padding for content
frame = ctk.CTkScrollableFrame(win, 
                               width=750, height=550, 
                               corner_radius=15, 
                               label_text = "Lung Disease Detection Center",
                               label_fg_color = "blue",
                               # label_font = ("Aptos Narrow",18),
                               # #border_width = 3,
                               # scrollbar_fg_color = "#E56565",
                               # scrollbar_button_color = "",
                               scrollbar_button_hover_color = "#E56565"
                              )
     
frame.pack(pady=20, padx=20)

# Name input field with enhanced styling
name_label = ctk.CTkLabel(frame, text="Enter Your Name (For Profile):", font=("Arial", 14))
name_label.pack(pady=10)

name_entry = ctk.CTkEntry(frame, width=300, font=("Arial", 14))
name_entry.pack(pady=10)

theme_switch = ctk.CTkSwitch(win, text="Dark Mode", command=toggle_theme)
theme_switch.place(relx=1.0, rely=1.0, anchor="se", x=-20, y=-20)  # Bottom-right with padding

# Location input field
location_label = ctk.CTkLabel(frame, text="Enter Your Location (City/Address):", font=("Arial", 14))
location_label.pack(pady=10)

location_entry = ctk.CTkEntry(frame, width=300, font=("Arial", 14))
location_entry.pack(pady=10)

# Create profile button with a better design
create_profile_button = ctk.CTkButton(frame, text="Create Profile", command=create_profile, width=250, height=40, font=("Arial", 14))
create_profile_button.pack(pady=15)

# Buttons organized in a row for better spacing
button_frame = ctk.CTkFrame(frame)
button_frame.pack(pady=20)

select_button = ctk.CTkButton(button_frame, text="Select X-Ray Image (For Registered Users)", command=predict_class, width=200, height=40)
select_button.grid(row=0, column=0, padx=20)

guest_button = ctk.CTkButton(button_frame, text="Select X-Ray Image (Guest Mode)", command=guest_mode, width=200, height=40)
guest_button.grid(row=0, column=1, padx=20)

image_label = ctk.CTkLabel(frame, text="", fg_color = "#564C5E", width=200, height=200)
image_label.pack(pady=10)

# Result display area with larger font
result_label = ctk.CTkLabel(frame, text="Predicted Class: ", font=("Arial", 16), wraplength=500)
result_label.pack(pady=10)

chatbot_button = ctk.CTkButton(frame, text="Chat with Bot 🤖", command=open_chatbot)
chatbot_button.pack(pady=10)  # Adjust position as needed

# History section
history_button = ctk.CTkButton(frame, text="View My History", command=view_history, width=200, height=40, font=("Arial", 14))
history_button.pack(pady=20)

history_label = ctk.CTkLabel(frame, text="Your History will be shown here.", font=("Arial", 12), wraplength=500)
history_label.pack(pady=10)

recommend_button = ctk.CTkButton(frame, text="Find Nearby Hospitals", command=show_recommendations, width=250, height=40, font=("Arial", 14))
recommend_button.pack(pady=15)

hospital_label = ctk.CTkLabel(frame, text="Hospital Recommendations will appear here.", font=("Arial", 14))
hospital_label.pack(pady=10)

show_map_button = ctk.CTkButton(frame, text="Show Nearby Hospitals Map", command=show_map, font=("Arial", 12))
show_map_button.pack(pady=20)

# Start the application
win.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 887ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
